# Jane Street Market Prediction - Submission
This is the actual notebook submitted to the competition. First, we load the trained models and use them to implement the `predict()` function.

In [ ]:
import json
import os

import pandas as pd

from tensorflow import keras

# load model
model_folder = os.path.join(os.pardir, "input", "jane-street-market-prediction-tpu")

with open(os.path.join(model_folder, "model.json"), "r") as f:
    model_json = f.read()

model = keras.models.model_from_json(model_json)
model.load_weights(os.path.join(model_folder, "model.h5"))

with open(os.path.join(model_folder, "params.json")) as file:
    params = json.loads(file.read())

window_size = params["window_size"]


# function to return predictions for the current trade
def predict(df):
    if len(df) < window_size:
        return 0

    series = df.iloc[-window_size:].to_numpy().reshape(1, 40, 130)
    pred = model.predict(series)
    return pred

Then we use the Jane Street API to make the predictions. In this setup the iterator `iter_test` yields single samples `test_df` in chronological order. The predict function needs to decide whether to accept or reject the trade at this time-point using only the current sample and the previous samples stored in `hist_df`. Set `pred_df` to 1 to accept the trade or to 0 to reject the trade.

In [ ]:
import janestreet

env = janestreet.make_env()
iter_test = env.iter_test()

df = pd.DataFrame()
last_date = -1

for (sample, pred) in iter_test:
    # if new date, clear df
    new_date = sample["date"].to_numpy() 
    if new_date != last_date:
        last_date = new_date
        df.drop(df.index, inplace=True)
        
    # append new sample to df
    features = sample[[c for c in sample.columns if "feature" in c]]
    df = df.append(features, ignore_index=True)
    
    # accept/reject current trade
    pred.action = predict(df)
    env.predict(pred)